In [ ]:
%matplotlib inline

In [ ]:
import gc
import pathlib
import sys

In [ ]:
import joblib
import lightgbm as lgb
import pandas as pd

In [ ]:
root_dir = "../.."

In [ ]:
root_dir_path = pathlib.Path(root_dir)
data_dir_path = root_dir_path / "data"
processed_dir_path = data_dir_path / "processed"
train_path = processed_dir_path / "train.parquet"
models_dir_path = root_dir_path / "models"
model_path = models_dir_path / "clf.joblib"
src_dir_path = root_dir_path / "src"

In [ ]:
sys.path.append(str(src_dir_path))

In [ ]:
from package.constants import *

In [ ]:
train = pd.read_parquet(train_path)

In [ ]:
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/135896
is_train = (train["date"] >= train_start_date) & (train["date"] < validation_start_date)
is_valid = train["date"] >= validation_start_date

In [ ]:
params = {
    "bagging_fraction": 0.75,
    "bagging_freq": 1,
    "feature_fraction": 0.8,
    "lambda_l2": 0.1,
    "learning_rate": 0.075,
    "metric": "binary_logloss",
    "min_data_in_leaf": 104,
    "n_jobs": -1,
    "num_leaves": 128,
    "objective": "binary",
    "seed": 0,
}

In [ ]:
is_train |= is_valid

In [ ]:
dtrain = lgb.Dataset(
    train.loc[is_train, features],
    train.loc[is_train, target] > 0.0,
    categorical_feature=categorical_features,
)

In [ ]:
%%time
model = lgb.train(params, dtrain, num_boost_round=1_250)

In [ ]:
ax = lgb.plot_importance(model, importance_type="split", figsize=(16, 9))

In [ ]:
ax = lgb.plot_importance(model, importance_type="gain", figsize=(16, 9))

In [ ]:
joblib.dump(model, model_path)